In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
# df = pd.read_csv("C:\\Users\\riffere\\OneDrive - Florida Panthers\\Documents\\LTP\\Kids_Club.csv")
# FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "kids_club_combined_data")

In [4]:
# q = """
# WITH initial as
#     (select
#         lower(parent_email) as parent_email,
#         case
#             when listagg(kc_group) like '%All-Star%' then 'All-Star'
#             when listagg(kc_group) like '%Pro%' then 'Pro'
#             else 'Rookie'
#         end as kc_group_final
#     from
#         custom.kids_club_combined_data
#     group by
#         lower(parent_email)),
# first_kids_club_season AS
#     (select
#         initial.parent_email, 
#         kc_group_final as kc_group, 
#         min(transaction_date) as transaction_date, 
#         min(season) as season
#     from
#         initial
#     left join
#         custom.kids_club_combined_data
#         on initial.parent_email = kids_club_combined_data.parent_email
#         and initial.kc_group_final = kids_club_combined_data.kc_group
#     group by
#         initial.parent_email, kc_group_final),
# hist_acct_ids AS
#     (SELECT
#         acct_id, 
#         lower(email_addr) as email, 
#         date(add_date) as add_date
#     FROM
#         custom.arch_final_v_customer),
# hist_emails_and_accounts AS
#     (SELECT
#          first_kids_club_season.parent_email, acct_id, CAST(MIN(season) AS VARCHAR) AS "season",
#          CAST(MIN(transaction_date) AS date) AS "transaction_date", add_date
#     FROM
#         first_kids_club_season
#     LEFT JOIN
#         hist_acct_ids on first_kids_club_season.parent_email = hist_acct_ids.email
#     WHERE
#         first_kids_club_season.parent_email is not null and first_kids_club_season.parent_email <> ''
#     GROUP BY
#         first_kids_club_season.parent_email, acct_id, add_date),
# historical_data AS
#     (SELECT
#         hist_emails_and_accounts.acct_id, parent_email, hist_emails_and_accounts.season, transaction_date, hist_emails_and_accounts.add_date,
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2017-18' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2017_18",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2018-19' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2018_19",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2019-20' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2019_20",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2020-21' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2020_21",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2021-22' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2021_22",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2022-23' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2022_23",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2017-18' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2017_18",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2018-19' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2018_19",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2019-20' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2019_20",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2020-21' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2020_21",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2021-22' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2021_22",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2022-23' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2022_23"
#     FROM
#         hist_emails_and_accounts
#     LEFT JOIN
#         custom.cth_historical_all_1718_2223 on hist_emails_and_accounts.acct_id = cth_historical_all_1718_2223.acct_id),
# historical_agg AS
#     (SELECT
#         parent_email, season, transaction_date, min(add_date) as add_date, max(is_stm_2017_18) as is_stm_2017_18,
#         max(is_stm_2018_19) as is_stm_2018_19,
#         max(is_stm_2019_20) as is_stm_2019_20, max(is_stm_2020_21) as is_stm_2020_21,
#         max(is_stm_2021_22) as is_stm_2021_22, max(is_stm_2022_23) as is_stm_2022_23,
#         sum(gross_rev_2017_18) as gross_rev_2017_18, sum(gross_rev_2018_19) as gross_rev_2018_19,
#         sum(gross_rev_2019_20) as gross_rev_2019_20, sum(gross_rev_2020_21) as gross_rev_2020_21,
#         sum(gross_rev_2021_22) as gross_rev_2021_22, sum(gross_rev_2022_23) as gross_rev_2022_23
#     FROM
#         historical_data
#     GROUP BY
#         parent_email, season, transaction_date),
# current_acct_ids AS
#     (SELECT
#          crm_id as acct_id, lower(email) as email, date(creation_datetime) as add_date
#     FROM
#         custom.seatgeek_v_clients),
# current_emails_and_accounts AS
#     (SELECT
#          first_kids_club_season.parent_email, acct_id, CAST(MIN(season) AS VARCHAR) AS "season",
#          CAST(MIN(transaction_date) AS date) AS "transaction_date", add_date
#     FROM
#         first_kids_club_season
#     LEFT JOIN
#         current_acct_ids on first_kids_club_season.parent_email = current_acct_ids.email
#     WHERE
#         first_kids_club_season.parent_email is not null and first_kids_club_season.parent_email <> ''
#     GROUP BY
#         first_kids_club_season.parent_email, acct_id, add_date),
# current_data AS
#     (SELECT
#          purch_client_crm_id as acct_id, parent_email, current_emails_and_accounts.season, current_emails_and_accounts.transaction_date,
#          current_emails_and_accounts.add_date, gross_revenue as gross_rev_2023_24,
#         CASE
#             WHEN "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2023_24"
#     FROM
#         current_emails_and_accounts
#     LEFT JOIN
#         custom.cth_v_ticket_2324 on cth_v_ticket_2324.purch_client_crm_id = current_emails_and_accounts.acct_id),
# current_agg AS
#     (SELECT
#         parent_email, season, transaction_date, min(add_date) as add_date,
#         max(is_stm_2023_24) as is_stm_2023_24, sum(gross_rev_2023_24) as gross_rev_2023_24
#     FROM
#         current_data
#     GROUP BY
#         parent_email, season, transaction_date),
# future_data AS
#     (SELECT
#          acct_id, parent_email, current_emails_and_accounts.season, current_emails_and_accounts.transaction_date,
#          current_emails_and_accounts.add_date, cth_v_ticket_subscription_2425.gross_revenue as gross_rev_2024_25,
#         CASE
#             WHEN cth_v_ticket_subscription_2425.ticket_type IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2024_25"
#     FROM
#         current_emails_and_accounts
#     LEFT JOIN
#         custom.cth_v_ticket_subscription_2425 on cth_v_ticket_subscription_2425.purch_client_crm_id = current_emails_and_accounts.acct_id),
# future_agg AS
#     (SELECT
#         parent_email, season, transaction_date, min(add_date) as add_date,
#         max(is_stm_2024_25) as is_stm_2024_25, sum(gross_rev_2024_25) as gross_rev_2024_25
#     FROM
#         future_data
#     GROUP BY
#         parent_email, season, transaction_date)
# SELECT
#     historical_agg.parent_email, kc_group, historical_agg.season,
#     historical_agg.transaction_date, coalesce(historical_agg.add_date, current_agg.add_date) as add_date,
#     is_stm_2017_18, is_stm_2018_19,
#     is_stm_2019_20, is_stm_2020_21, is_stm_2021_22, is_stm_2022_23, is_stm_2023_24, is_stm_2024_25,
#     gross_rev_2017_18, gross_rev_2018_19, gross_rev_2019_20,
#     gross_rev_2020_21, gross_rev_2021_22, gross_rev_2022_23, gross_rev_2023_24, gross_rev_2024_25
# FROM
#     historical_agg
# LEFT JOIN
#     current_agg on historical_agg.parent_email = current_agg.parent_email
# LEFT JOIN
#     future_agg on historical_agg.parent_email = future_agg.parent_email
# LEFT JOIN
#     first_kids_club_season on historical_agg.parent_email = first_kids_club_season.parent_email
# ORDER BY
#     parent_email
# """

In [11]:
q = """
WITH initial as
    (select
        lower(parent_email) as parent_email,
        case
            when listagg(kc_group) like '%All-Star%' then 'All-Star'
            when listagg(kc_group) like '%Pro%' then 'Pro'
            else 'Rookie'
        end as kc_group_final
    from
        custom.kids_club_combined_data
    group by
        lower(parent_email)),
first_kids_club_season AS
    (select
        initial.parent_email,
        kc_group_final as kc_group,
        min(transaction_date) as transaction_date,
        min(season) as season
    from
        initial
    left join
        custom.kids_club_combined_data
        on lower(initial.parent_email) = lower(kids_club_combined_data.parent_email)
    group by
        initial.parent_email, kc_group_final),
acct_rev_info AS
    (SELECT
        contacts.email,
        createdon as add_date,
        CASE
            WHEN ticket.season like '2021-22' then gross_revenue
            ELSE 0
        END AS "gross_rev_2122",
        CASE
            WHEN ticket.season like '2022-23' then gross_revenue
            ELSE 0
        END AS "gross_rev_2223",
        CASE
            WHEN ticket.season like '2023-24' then gross_revenue
            ELSE 0
        END AS "gross_rev_2324",
        CASE
            WHEN ticket.season like '2024-25' then gross_revenue
            ELSE 0
        END AS "gross_rev_2425",
        CASE
            WHEN ticket.season like '2021-22' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2122",
        CASE
            WHEN ticket.season like '2022-23' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2223",
         CASE
            WHEN ticket.season like '2023-24' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2324",
        CASE
            WHEN ticket.season like '2024-25' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2425"
    FROM
        custom.cth_v_historical_ticket ticket
    JOIN
        custom.korepss_externalsystemtocontact ext ON ticket.purchaser_ticketing_id = ext.externalcontactid
    JOIN
        custom.korepss_contacts contacts ON ext.crmcontactid = contacts.contactid),
acct_rev_agg as
    (SELECT
        email,
        min(add_date) as add_date,
        sum(gross_rev_2122) as gross_rev_2122,
        sum(gross_rev_2223) as gross_rev_2223,
        sum(gross_rev_2324) as gross_rev_2324,
        sum(gross_rev_2425) as gross_rev_2425,
        max(is_stm_2122) as is_stm_2122,
        max(is_stm_2223) as is_stm_2223,
        max(is_stm_2324) as is_stm_2324,
        max(is_stm_2425) as is_stm_2425
    FROM
        acct_rev_info
    group by
        email)
SELECT
    parent_email,
    kc_group,
    transaction_date::date as transaction_date,
    first_kids_club_season.season,
    add_date::date as add_date,
    coalesce(gross_rev_2122,0) as gross_rev_2122,
    coalesce(gross_rev_2223,0) as gross_rev_2223,
    coalesce(gross_rev_2324,0) as gross_rev_2324,
    coalesce(gross_rev_2425,0) as gross_rev_2425,
    coalesce(is_stm_2122,0) as is_stm_2122,
    coalesce(is_stm_2223,0) as is_stm_2223,
    coalesce(is_stm_2324,0) as is_stm_2324,
    coalesce(is_stm_2425,0) as is_stm_2425
FROM
    first_kids_club_season
LEFT JOIN
    acct_rev_agg on first_kids_club_season.parent_email = acct_rev_agg.email
WHERE
    first_kids_club_season.parent_email is not null and first_kids_club_season.parent_email <> ''
"""

In [12]:
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [13]:
df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)
df['add_date'] = pd.to_datetime(df['add_date']).dt.date
df['transaction_date'] = pd.to_datetime(df['transaction_date']).dt.date

C:\Users\riffere\AppData\Local\Temp\ipykernel_25212\4015671550.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)


In [14]:
df['pre_kids_club_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] < row['transaction_date'] else 0), axis = 1)
df['post_kids_club_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] >= row['transaction_date'] else 0), axis = 1)

In [15]:
season_dict = {
    "2021-22" : 1,
    "2022-23" : 2,
    "2023-24" : 3,
    "2024-25" : 4
}

# sum season tickets before/after
for key, value in season_dict.items():
    col_list = list(df.columns[5:(5+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_kids_club_revenue'] =  df[col_list].sum(axis = 1)

for key, value in season_dict.items():
    col_list = list(df.columns[(4 + value):9])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_kids_club_revenue'] =  df[col_list].sum(axis = 1)

# sum revenue before/after
for key, value in season_dict.items():
    col_list = list(df.columns[9:(9+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_kids_club_tickets'] =  df[col_list].sum(axis = 1)


for key, value in season_dict.items():
    col_list = list(df.columns[(8 + value):13])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_kids_club_tickets'] =  df[col_list].sum(axis = 1)

In [16]:
df

,parent_email,kc_group,transaction_date,season,add_date,gross_rev_2122,gross_rev_2223,gross_rev_2324,gross_rev_2425,is_stm_2122,is_stm_2223,is_stm_2324,is_stm_2425,pre_kids_club_email,post_kids_club_email,pre_kids_club_revenue,post_kids_club_revenue,pre_kids_club_tickets,post_kids_club_tickets
0,nkamsalem@gmail.com,Rookie,2023-01-27,2022-23,2018-01-22,5278.41,12048.04,12754.22,5778.0,1,1,1,1,1,0,17326.45,30580.26,2.0,3.0
1,candicehallack@gmail.com,All-Star,2022-10-16,2021-22,2021-02-18,249.00,0.00,0.00,0.0,0,0,0,0,1,0,249.00,249.00,0.0,0.0
2,do@opplawfirm.com,Rookie,2021-11-22,2021-22,2018-01-18,2161.98,0.00,0.00,0.0,1,0,0,0,1,0,2161.98,2161.98,1.0,1.0
3,mecca0912@gmail.com,Rookie,2024-04-04,2023-24,2018-01-18,0.00,0.00,0.00,0.0,0,0,0,0,1,0,0.00,0.00,0.0,0.0
4,colomboj@floridapanthers.com,All-Star,2022-10-20,2022-23,2018-01-18,0.00,0.00,0.00,0.0,0,0,0,0,1,0,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2788,rscholink11@yahoo.com,Rookie,2023-01-13,2022-23,2030-01-01,0.00,0.00,0.00,0.0,0,0,0,0,0,1,0.00,0.00,0.0,0.0
2789,sharonstix@bellsouth.net,Rookie,2022-01-04,2021-22,2030-01-01,0.00,0.00,0.00,0.0,0,0,0,0,0,1,0.00,0.00,0.0,0.0
2790,thomaszur@yahoo.com,Rookie,2023-02-28,2022-23,2030-01-01,0.00,0.00,0.00,0.0,0,0,0,0,0,1,0.00,0.00,0.0,0.0
2791,trishammcdonald@gmail.com,Rookie,2024-03-12,2023-24,2030-01-01,0.00,0.00,0.00,0.0,0,0,0,0,0,1,0.00,0.00,0.0,0.0


In [15]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "kids_club_type_data")